In [1]:
import os
import torch
import cv2
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import torch
import numpy as np

# Function to calculate intersection over union between two bounding boxes
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set device to use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set confidence threshold for detections
conf_threshold = 0.5

# Set input and output folder paths
input_folder_path = '/content/drive/MyDrive/Mahesh/A'
output_folder_path = '/content/drive/MyDrive/Mahesh/output_folder'

# Create output folder if it does not exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop over images in input folder
for filename in os.listdir(input_folder_path):
    # Check if file is an image
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load image and run YOLOv5 model
        image_path = os.path.join(input_folder_path, filename)
        image = cv2.imread(image_path)
        results = model(image)
        results_np = np.array(results.xyxy[0]) 
        if 0 in results_np[:,5]:
            # Loop over detections
            for detection in results_np:
                if detection[5] == 0:  # If class is pedestrian
                    x1, y1, x2, y2, conf, class_id = detection.tolist()
                    # Draw bounding box around detected pedestrian
                    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label_file_path = os.path.join(input_folder_path, filename[:-4] + '.txt')
        if os.path.exists(label_file_path):
            with open(label_file_path, 'r') as label_file:
                label_lines = label
                                # Extract class and bounding box coordinates from label line
                label = label_lines.strip().split()
                class_label = int(label[0])
                x_min, y_min, x_max, y_max = map(float, label[1:])
                # Convert label coordinates to pixel values
                img_height, img_width, _ = image.shape
                x_min_pixel = int(x_min * img_width)
                y_min_pixel = int(y_min * img_height)
                x_max_pixel = int(x_max * img_width)
                y_max_pixel = int(y_max * img_height)
                # Calculate intersection over union between ground truth and detected bounding boxes
                iou = bb_intersection_over_union((x_min_pixel, y_min_pixel, x_max_pixel, y_max_pixel), (x1, y1, x2, y2))
                # If intersection over union is high, draw ground truth bounding box in red
                if iou > 0.5:
                    cv2.rectangle(image, (x_min_pixel, y_min_pixel), (x_max_pixel, y_max_pixel), (0, 0, 255), 2)
                if len(label_lines.split()) == 5:
                        class_name, x, y, w, h = label_lines.split()
                        # If class is pedestrian
                        if class_name == '0':
                            # Convert label coordinates to pixel values
                            img_height, img_width, _ = image.shape
                            x1_gt = int((float(x) - float(w) / 2) * img_width)
                            y1_gt = int((float(y) - float(h) / 2) * img_height)
                            x2_gt = int((float(x) + float(w) / 2) * img_width)
                            y2_gt = int((float(y) + float(h) / 2) * img_height)
                            # Calculate intersection over union between detected and ground truth bounding boxes
                            iou = bb_intersection_over_union([x1, y1, x2, y2], [x1_gt, y1_gt, x2_gt, y2_gt])
                            # If intersection over union is high, draw ground truth bounding box
                            if iou > 0.5:
                                cv2.rectangle(image, (x1_gt, y1_gt), (x2_gt, y2_gt), (255, 0, 0), 2)

    output_path = os.path.join(output_folder_path, filename)
    cv2.imwrite(output_path, image)





/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-17 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 33.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
